# Merge IRS Data years 2021 and 2011 and calculate the Delta between them.

## 1. Load both years

In [3]:
#Libraries and Settings
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',10)

In [4]:
#Data File to Dataframe
file='/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data/1_Load_IRS_2011.csv'
irs11=pd.read_csv(file, converters={'zip': str})
print(irs11.shape)
irs11.head()

(27838, 9)


,zip,state,total_returns,agi_stub_1_ratio,agi_stub_2_ratio,agi_stub_3_ratio,agi_stub_4_ratio,agi_stub_5_ratio,agi_stub_6_ratio
0,01001,MA,8655.0,0.507683,0.264240,0.000000,0.103293,0.112998,0.011785
1,01002,MA,10168.0,0.419945,0.179485,0.120378,0.083596,0.143588,0.053009
2,01005,MA,2201.0,0.330304,0.228987,0.169468,0.127669,0.133121,0.010450
3,01007,MA,7225.0,0.323599,0.200000,0.154464,0.116125,0.180069,0.025744
4,01008,MA,646.0,0.304954,0.226006,0.164087,0.153251,0.151703,0.000000


In [5]:
#Data File to Dataframe
file='/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data/1_Load_IRS_2021.csv'
irs21=pd.read_csv(file, converters={'zip': str})
print(irs21.shape)
irs21.head()

(27654, 15)


,zip,state,total_returns,agi_stub_1_ratio,agi_stub_2_ratio,agi_stub_3_ratio,agi_stub_4_ratio,agi_stub_5_ratio,agi_stub_6_ratio,agi_stub_1_income,agi_stub_2_income,agi_stub_3_income,agi_stub_4_income,agi_stub_5_income,agi_stub_6_income
0,01001,MA,9030.0,0.254707,0.232558,0.189369,0.115172,0.172757,0.035437,30325.0000,79712.0,106034.0,91146.0000,210887.0,116603.0000
1,01002,MA,9720.0,0.268519,0.207819,0.118313,0.083333,0.197531,0.124486,31643.0000,73567.0,71644.0,71058.0000,278260.0,567106.0000
2,01005,MA,2500.0,0.236000,0.228000,0.164000,0.124000,0.212000,0.036000,7483.0000,21180.0,25765.0,26745.0000,71615.0,33306.0000
3,01007,MA,7990.0,0.236546,0.185232,0.143930,0.111389,0.251564,0.071339,23259.0000,55073.0,71881.0,78044.0000,278063.0,196478.0000
4,01008,MA,620.0,0.193548,0.209677,0.161290,0.145161,0.241935,0.048387,1469.0001,4843.0,6610.0,7648.0001,20536.0,10088.0001


### 1.1. Number of lines Verification
The number of lines (one per zip code) in 2021 is different from 2011. Let's take a look at what is happening, how to move forward and what is the potential impact.

In [6]:
zip_list_21=irs21['zip'].tolist()
zip_list_11=irs11['zip'].tolist()

total_zips_11=irs21['zip'].nunique()
total_zips_21=irs11['zip'].nunique()


#what is in 21 that is not in 11:
out_of_list_11 = [item for item in zip_list_21 if item not in zip_list_11]
p_out_of_21=len(out_of_list_11)/total_zips_21*100
print('number of zips in 21 that are not in 11: ',len(out_of_list_11),'which represents: ','{:.2f}'.format(p_out_of_21),'% of total')

#what is in 11 that is not in 21:
out_of_list_21 = [item for item in zip_list_11 if item not in zip_list_21]
p_out_of_11=len(out_of_list_21)/total_zips_11*100
print('number of zips in 11 that are not in 21: ',len(out_of_list_21),'which represents: ', '{:.2f}'.format(p_out_of_11),'% of total')
#Let's see a few examples
#print(out_of_list_21)


number of zips in 21 that are not in 11:  97 which represents:  0.35 % of total
number of zips in 11 that are not in 21:  281 which represents:  1.02 % of total


In [7]:
#For optional evaluation

#zip_codes_missing_in_2011=irs21[(irs21['zip'].isin(out_of_list_11) )]
#zip_codes_missing_in_2021=irs11[(irs11['zip'].isin(out_of_list_21) )]

### Reasons why some Zip codes are not present in both years
**Administrative Changes:** ZIP codes are not officially defined by the U.S. Census Bureau but are assigned by the United States Postal Service (USPS). Administrative changes, such as the creation, consolidation, or elimination of ZIP codes, may occur over time due to changes in postal delivery routes or administrative boundaries.

**Population Shifts:** Changes in population distribution and density within a region can influence the need for postal services and the assignment of ZIP codes. If the population in the area corresponding to ZIP code 04343 decreased significantly between 2011 and 2021, USPS might have adjusted postal delivery routes, resulting in the elimination of the ZIP code.

**Postal Service Updates:** The USPS regularly reviews and updates postal delivery routes to optimize efficiency and service coverage. This process may lead to changes in ZIP code boundaries or the elimination of less-used ZIP codes.

**Data Reporting:** Discrepancies in ZIP code data between censuses can also arise from differences in data collection methodologies, reporting practices, and data processing techniques used by the Census Bureau and other agencies.



## 2. Merge the 2 Dataframes

In [8]:
#Using outer to keep the zips that exist only in year but not in the other. For those there will be NaNs.
#Using the suffixes _21 and _11 to designate the respective years.
irs=pd.merge(irs21,irs11,how='outer', on=['zip','state'], suffixes=('_21', '_11') )
irs.shape


(27935, 22)

### 2.1. Reorder the merged fields

In [9]:
# I don't like the order. I will change it
l_irs21=irs21.columns.tolist()
l_irs11=irs11.columns.tolist()


l_irs21_diff=set(l_irs21)-set(l_irs11)
#print(l_irs21_diff)

#zip is the first and only common non duplicated field.
l_irs = ['zip','state']
for item1, item2 in zip(l_irs21[2:], l_irs11[2:]):
    l_irs.append(item1+'_21')
    l_irs.append(item2+'_11')

#adding the columns that are only available in 21
l_irs = l_irs + list(l_irs21_diff)

#use the combined list
#print(l_irs)
irs=irs[l_irs]


## 3. Calculate Deltas

In [10]:

#First field is 'zip' and there is nothing to calculate for it.
#I take irs11 because it has less fields and those are the ones in common.
all_common_fields=irs11.columns.tolist()[2:]

print('BEFORE shape: ',irs.shape)

for item in all_common_fields:
    column_21=item+'_21'
    column_11=item+'_11'
    new_column='delta_'+item
    #I don't have to worry about either columns containing nans because the result will be NaN and I can treat the result directly.
    irs[new_column] = irs[column_21] - irs[column_11]
print('AFTER adding Delta columns shape: ',irs.shape)

l_irs = ['zip','state']
for item in all_common_fields:
    column_21=item+'_21'
    column_11=item+'_11'
    new_column='delta_'+item
    l_irs.append(column_21)
    l_irs.append(column_11)
    l_irs.append(new_column)

# I use the set of different fields that exist only in 21 and put them at the end
l_irs = l_irs + list(l_irs21_diff)

irs=irs[l_irs]

BEFORE shape:  (27935, 22)
AFTER adding Delta columns shape:  (27935, 29)


### 4. Save the Changes

In [11]:
#After all the changes, let's save in a csv file.

import os
outname = '2_Org_Irs.csv'
outdir = '/Users/c32/Documents/NYCDSA/Projects/DATA/Ready_Data'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)    

irs.to_csv(fullname, header=True, index=False)
print("Saved!")

Saved!
